In [ ]:
import math

def calc_x_reserves(APY,y_reserves,days_until_maturity,time_stretch):
    t=days_until_maturity/(365*time_stretch)
    T=days_until_maturity/365
    return y_reserves*(-(2/((1-T*APY/100)**(1/t)-1))-2)

def calc_price(x_reserves,y_reserves,total_supply,days_until_maturity,time_stretch):
  t=days_until_maturity/(365*time_stretch)
  return 1/((y_reserves + total_supply)/x_reserves)**t

def calc_k(in_reserves,out_reserves,t):
  return pow(in_reserves,1-t) + pow(out_reserves,1-t)

def calc_apy(price,days_until_maturity):
  T=days_until_maturity/365
  return (1-price)/T * 100

def is_trade_valid(in_,in_reserves,out_reserves,t):
  k = pow(in_reserves,1-t) + pow(out_reserves,1-t)
  check = math.log(k)/math.log(in_reserves + in_) + t
  return True if check >= 1 else False

def calc_max_trade(in_reserves,out_reserves,t):
  k = pow(in_reserves,1-t) + pow(out_reserves,1-t)
  return k**(1/(1-t)) - in_reserves

def calc_out_given_in(in_,in_reserves,out_reserves,token_out,g,t):
        k=pow(in_reserves,1-t) + pow(out_reserves,1-t)
        without_fee = out_reserves - pow(k-pow(in_reserves+in_,1-t),1/(1-t))
        if token_out == "base":
            fee =  (in_-without_fee)*g
            with_fee = without_fee-fee
        elif token_out == "fyt":
            fee =  (without_fee-in_)*g
            with_fee = without_fee-fee   
        return (with_fee,without_fee,fee)
    

In [ ]:
APY = 25
days_until_maturity = 90
time_stretch = 4.43
y_reserves = 100
x_reserves = calc_x_reserves(APY,y_reserves,days_until_maturity,time_stretch)
print("(APY,y_reserves,days_until_maturity,time_stretch): "+str((APY,y_reserves,days_until_maturity,time_stretch)))
print("required x_reserves: " + str(x_reserves))
total_supply=x_reserves+y_reserves
price=calc_price(x_reserves,y_reserves,total_supply,days_until_maturity,time_stretch)
print("resulting price: " + str(price))
print("check apy: " + str(calc_apy(price,days_until_maturity)))

amount=106
g=0.1
t=days_until_maturity/(365*time_stretch)
(with_fee,without_fee,fee)=calc_out_given_in(amount,y_reserves+total_supply,x_reserves,"base",g,t)
without_slippage_or_fee = price * amount
slippage = abs(without_slippage_or_fee-without_fee)
print("\nTrade size: " + str(amount))
print("Price w/o slippage: " + str(without_slippage_or_fee))
print("Price with slippage: " + str(without_fee))
print("Price with slippage and fee: " + str(with_fee))
print("Total fee: " + str(fee))
print("Percent slippage: " + str(slippage/without_slippage_or_fee * 100))
print("check apy: " + str(calc_apy(with_fee/amount,days_until_maturity)))
print("x_reserves = {:} vs price_with_slippage_fee = {:}".format(x_reserves, with_fee))
print("is valid trade = {:}".format(is_trade_valid(amount,y_reserves+total_supply,x_reserves,t)))
print("max valid trade = {:}".format(calc_max_trade(y_reserves+total_supply,x_reserves,t)))

amount=95
g=0.1
t=days_until_maturity/(365*time_stretch)

(with_fee,without_fee,fee)=calc_out_given_in(amount,x_reserves,y_reserves+total_supply,"fyt",g,t)
price=calc_price(x_reserves+amount,y_reserves-with_fee,total_supply,days_until_maturity,time_stretch)
without_slippage_or_fee = price * amount
slippage = abs(without_slippage_or_fee-without_fee)
print("\nTrade size: " + str(amount))
print("Price w/o slippage: " + str(without_slippage_or_fee))
print("Price with slippage: " + str(without_fee))
print("Price with slippage and fee: " + str(with_fee))
print("Total fee: " + str(fee))
print("Percent slippage: " + str(slippage/without_slippage_or_fee * 100))

print("check apy: " + str(calc_apy(price,days_until_maturity)))
print("x_reserves = {:} vs price_with_slippage_fee = {:}".format(x_reserves, with_fee))
print("is valid trade = {:}".format(is_trade_valid(amount,y_reserves+total_supply,x_reserves,t)))





In [ ]:
APY = 1
tranche_length = 30
days_until_maturity = tranche_length
time_stretch = 5
y_reserves = 50
x_reserves = calc_x_reserves(APY,y_reserves,tranche_length,time_stretch)
total_supply=x_reserves+y_reserves
print("required x_reserves: " + str(x_reserves))
g=0.05
t=days_until_maturity/(365*time_stretch)
k=calc_k(y_reserves+total_supply,x_reserves,t)
print("k: " + str(k))
print("k^(1/(1-t)): " + str(k**(1/(1-t))))
max_trade = calc_max_trade(y_reserves+total_supply,x_reserves,t) - .0001
print("max valid trade = {:}".format(max_trade))
(with_fee,without_fee,fee)=calc_out_given_in(max_trade,y_reserves+total_supply,x_reserves,"base",g,t)
print("Total Price after max trade: " + str(without_fee))
unit_price = without_fee/max_trade
print("Unit Price after max trade: " + str(unit_price))
resulting_apy=calc_apy(unit_price,days_until_maturity)
print("resulting apy: " + str(resulting_apy))
(resulting_apy-APY)/APY * 100


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
fig, ax = plt.subplots(figsize=(12,12))
time_stretch=1

for tranche_length in [30,60,120]:
    apy_data = []
    x_reserves_data = []
    for APY in np.arange(1, 51, 1):
      days_until_maturity = tranche_length
      y_reserves=100
      x_reserves = calc_x_reserves(APY,y_reserves,days_until_maturity,time_stretch)
      apy_data.append(APY)
      x_reserves_data.append(x_reserves/y_reserves)

    x=apy_data
    y=x_reserves_data
    ax.set_xlabel("PT APY",fontsize=18)
    ax.set_ylabel("Base Reserves to PT Reserves Ratio",fontsize=18)
    ax.set_xticks(np.arange(0, 51, 1))
    ax.set_ylim(0,50)
    ax.set_yticks(np.arange(0, 51, 5))
    #ax.set_yticks(np.arange(0, 105, 5))
    ax.plot(x, y,label=tranche_length)
    ax.legend(title="Term Length")
plt.title("Reserves Ratio Vs PT APY",fontsize=20 )
plt.grid(b=True, which='major', color='#666666', linestyle='-',alpha=0.2)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
fig, ax = plt.subplots(figsize=(12,12))

for time_stretch in [1,3,5,10,20,30,40]:
    apy_data = []
    x_reserves_data = []
    for APY in np.arange(1, 51, 1):
      days_until_maturity = 30
      y_reserves=100
      x_reserves = calc_x_reserves(APY,y_reserves,days_until_maturity,time_stretch)
      apy_data.append(APY)
      x_reserves_data.append(x_reserves/y_reserves)

    x=apy_data
    y=x_reserves_data
    ax.set_xlabel("PT APY",fontsize=18)
    ax.set_ylabel("Base Reserves to PT Reserves Ratio",fontsize=18)
    ax.set_xticks(np.arange(0, 51, 1))
    ax.set_ylim(0,10)
    ax.set_yticks(np.arange(0, 11, 1))
    ax.plot(x, y,label=time_stretch)
    ax.legend(title="Time Stretch")
plt.title("Reserves Ratio Vs PT APY",fontsize=20 )
plt.grid(b=True, which='major', color='#666666', linestyle='-',alpha=0.2)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

APY = 20
days_until_maturity = 90
time_stretch = 5
y_reserves = 100
x_reserves = calc_x_reserves(APY,y_reserves,days_until_maturity,time_stretch)
total_supply=x_reserves+y_reserves
epsilon=.0000001

times = []
max_trade_amounts = []

for t in np.arange(328, 0, -.1):
  times.append(t/365)
  max_trade = calc_max_trade(y_reserves+total_supply,x_reserves,t/365)-epsilon
  max_trade_amounts.append(max_trade)


fig, ax = plt.subplots(figsize=(12,12))

x=times
y=max_trade_amounts
ax.set_title("Max Allowable Input FYT Trade Size vs t-Param\n(base_reserves,fyt_reserves)=({:.2f},{:.2f})".format(x_reserves,y_reserves),fontsize=18)
ax.set_xlabel("t",fontsize=18)
ax.set_ylabel("Max Input FYT Trade Size",fontsize=18)
ax.invert_xaxis()
ax.plot(x, y)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
APY = 20
days_until_maturity = 90
time_stretch = 5
y_reserves = 100
x_reserves = calc_x_reserves(APY,y_reserves,days_until_maturity,time_stretch)
total_supply=x_reserves+y_reserves
epsilon=.0000001
g=.1
times = []
days_until_maturity_list = []
max_trade_amounts = []
resulting_apys = []

for t in np.arange(days_until_maturity, 0, -.1):
  times.append(t/(365*time_stretch))
  days_until_maturity_list.append(t)
  max_trade = calc_max_trade(y_reserves+total_supply,x_reserves,t/(365*time_stretch))-epsilon
  max_trade_amounts.append(max_trade)
  (with_fee,without_fee,fee)=calc_out_given_in(max_trade,y_reserves+total_supply,x_reserves,"base",g,t/(365*time_stretch))
  resulting_apy=calc_apy(without_fee/max_trade,t)
  resulting_apys.append((resulting_apy-APY)/APY * 100)

fig, ax1 = plt.subplots(figsize=(12,12))
ax2 = ax1.twinx()

x = days_until_maturity_list
y1 = max_trade_amounts
ax1.set_title("Days Until Maturity vs Max Allowable Input FYT Trade Size vs Max % Change in Implied APY\n\n(base_reserves, fyt_reserves)=({:.2f}, {:.2f})\ntime_stretch={:}".format(x_reserves,y_reserves,time_stretch),fontsize=18)
ax1.set_xlabel("Days Until Maturity",fontsize=18)
ax1.set_ylabel("Max Input FYT Trade Size",fontsize=18,color='b')
ax1.invert_xaxis()
ax1.plot(x, y1,'b-')

y2 = resulting_apys
ax2.set_ylabel("Max % Change in Implied APY",fontsize=18,color='g')
ax2.plot(x, y2,'g-')

In [ ]:

import matplotlib.pyplot as plt
import numpy as np

for time_stretch in [20,30,40]:
  fig, ax1 = plt.subplots(figsize=(12,12))
  APY = 1.7
  days_until_maturity = 180
  y_reserves = 50
  x_reserves = calc_x_reserves(APY,y_reserves,days_until_maturity,time_stretch)
  total_supply=x_reserves+y_reserves
  epsilon=.0000001
  g=.1
  times = []
  days_until_maturity_list = []
  max_trade_amounts = []
  resulting_apys = []

  for t in np.arange(days_until_maturity, 0, -.1):
    times.append(t/(365*time_stretch))
    days_until_maturity_list.append(t)
    max_trade = calc_max_trade(y_reserves+total_supply,x_reserves,t/(365*time_stretch))-epsilon
    max_trade_amounts.append(max_trade)
    (with_fee,without_fee,fee)=calc_out_given_in(max_trade,y_reserves+total_supply,x_reserves,"base",g,t/(365*time_stretch))
    resulting_apy=calc_apy(without_fee/max_trade,t)
    resulting_apys.append((resulting_apy-APY)/APY * 100)


  ax2 = ax1.twinx()

  x = days_until_maturity_list
  y1 = max_trade_amounts
  ax1.set_title("Days Until Maturity vs Max Allowable Input FYT Trade Size vs Max % Change in Implied APY\n\nAPY={:.2f}%\n(base_reserves, fyt_reserves)=({:.2f}, {:.2f})\ntime_stretch={:}".format(APY,x_reserves,y_reserves,time_stretch),fontsize=18)
  ax1.set_xlabel("Days Until Maturity",fontsize=18)
  ax1.set_ylabel("Max Input FYT Trade Size",fontsize=18,color='b')
  ax1.invert_xaxis()
  y2 = resulting_apys
  ax2.set_ylabel("Max % Change in Implied APY",fontsize=18,color='g')

  if time_stretch == 20: 
    ax1.plot(x, y1,'b-')
    ax2.plot(x, y2,'g-')
  elif time_stretch == 30:
    ax1.plot(x, y1,'b--')
    ax2.plot(x, y2,'g--')
  elif time_stretch == 40:
    ax1.plot(x, y1,'b.')
    ax2.plot(x, y2,'g.')




In [ ]:

import matplotlib.pyplot as plt
import numpy as np

fig, ax1 = plt.subplots(figsize=(12,12))

min_apy_changes = []
max_apy_changes = []
time_stretches = []
for time_stretch in np.arange(1, 51, 1):
  APY = 1.7
  days_until_maturity = 180
  y_reserves = 50
  x_reserves = calc_x_reserves(APY,y_reserves,days_until_maturity,time_stretch)
  total_supply=x_reserves
  epsilon=.0000001
  g=.1

  times = []
  days_until_maturity_list = []
  max_trade_amounts = []
  resulting_apys = []

  for t in np.arange(days_until_maturity, 0, -.1):
    times.append(t/(365*time_stretch))
    days_until_maturity_list.append(t)
    max_trade = calc_max_trade(y_reserves+total_supply,x_reserves,t/(365*time_stretch))-epsilon
    max_trade_amounts.append(max_trade)
    (with_fee,without_fee,fee)=calc_out_given_in(max_trade,y_reserves+total_supply,x_reserves,"base",g,t/(365*time_stretch))
    resulting_apy=calc_apy(without_fee/max_trade,t)
    resulting_apys.append((resulting_apy-APY)/APY * 100)

  time_stretches.append(time_stretch)
  min_apy_changes.append(min(resulting_apys))
  max_apy_changes.append(max(resulting_apys))

ax1.scatter(time_stretches,min_apy_changes,color='g')
ax1.scatter(time_stretches,max_apy_changes,color='r')
#ax1.set_ylim(0,1000)

# this plot tells me that the difference between the min and max % change apy isnt very much

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

cols = ["apy","tranche_length","time_stretch",\
        "x_reserves", "y_reserves",\
        "min_k","max_k",\
        "min_trade_amount","max_trade_amount",\
        "min_resulting_fyt_unit_prices","max_resulting_fyt_unit_prices",\
        "min_resulting_apy","max_resulting_apy",\
        "min_apy_change","max_apy_change"]


tests = []
g=.1
for y_reserves in np.arange(50, 1010, 50):
  for tranche_length in np.arange(30, 181, 30):
    for APY in np.arange(1, 51, 1):
      for time_stretch in np.arange(1, 40.1, .1):
        days_until_maturity = tranche_length
        x_reserves = calc_x_reserves(APY,y_reserves,days_until_maturity,time_stretch)
        total_supply=x_reserves+y_reserves
        epsilon=.0000001

        times = []
        days_until_maturity_list = []
        ks = []
        max_trade_amounts = []
        resulting_fyt_unit_prices = []
        resulting_apys = []
        resulting_apy_changes = []

        for day in np.arange(days_until_maturity, 0, -1):
          times.append(day/(365*time_stretch))
          days_until_maturity_list.append(day)
          k=calc_k(y_reserves+total_supply,x_reserves,day/(365*time_stretch))
          ks.append(k)
          max_trade = calc_max_trade(y_reserves+total_supply,x_reserves,day/(365*time_stretch))-epsilon
          max_trade_amounts.append(max_trade)
          (with_fee,without_fee,fee)=calc_out_given_in(max_trade,y_reserves+total_supply,x_reserves,"base",g,day/(365*time_stretch))
          resulting_fyt_unit_price = without_fee/max_trade
          resulting_fyt_unit_prices.append(resulting_fyt_unit_price)
          resulting_apy=calc_apy(resulting_fyt_unit_price,day)
          resulting_apys.append(resulting_apy)
          resulting_apy_changes.append((resulting_apy-APY)/APY * 100)

        test = [APY,tranche_length,time_stretch,x_reserves,y_reserves,\
                min(ks),max(ks),\
                min(max_trade_amounts),max(max_trade_amounts),\
                min(resulting_fyt_unit_prices),max(resulting_fyt_unit_prices),\
                min(resulting_apys), max(resulting_apys),\
                min(resulting_apy_changes),max(resulting_apy_changes)]
        tests.append(test)

df = pd.DataFrame(tests,columns=cols)
#df.to_csv('apy_change.csv')
df

In [ ]:
df.to_csv('apy_change_new.csv')

In [ ]:
import pandas as pd
df = pd.read_csv('apy_change_new.csv') 
df = df.drop(columns=['Unnamed: 0']).reset_index()
cols = df.columns
for col in cols:
    df[col] = df[col].astype(float)
df.head()

In [ ]:
import numpy as np
#df.to_csv('apy_change.csv')
pd.set_option('display.max_rows', None)
df['reserve_ratio']=df['x_reserves']/df['y_reserves']

df['time_stretch'] = round(df['time_stretch'],5)

reserve_ratio_filter=(df['reserve_ratio']>=.1)&(df['reserve_ratio']<=1)
price_discovery_filter=(df['max_apy_change']>=50)
df_filtered = df[reserve_ratio_filter & price_discovery_filter].reset_index()

apy_t_stretches=[]
for APY in np.arange(3, 51, 1):
    min_ts=df_filtered[df_filtered['apy']==APY]['time_stretch'].min()
    max_ts=df_filtered[df_filtered['apy']==APY]['time_stretch'].max()
    min_tl=df_filtered[(df_filtered['apy']==APY)&(df_filtered['time_stretch']>=min_ts)&(df_filtered['time_stretch']<=max_ts)]['tranche_length'].min()
    max_tl=df_filtered[(df_filtered['apy']==APY)&(df_filtered['time_stretch']>=min_ts)&(df_filtered['time_stretch']<=max_ts)]['tranche_length'].max()
    apy_t_stretches.append((APY,min_ts,max_ts,min_tl,max_tl))
     

In [ ]:
import numpy as np

pd.reset_option("display.max_rows")
df['reserve_ratio']=df['x_reserves']/df['y_reserves']
df['spot_price']=(df['x_reserves']/(2*df['y_reserves']+df['x_reserves']))**(df['tranche_length']/(365*df['time_stretch']))
df['spot_apy']=(1-df['spot_price'])/(df['tranche_length']/365) * 100


# we want to target the time stretch that allows the reserve ratio to be equal to the spot price
reserve_ratio_filter=(df['reserve_ratio']-df['reserve_ratio']*0.05<=df['spot_price'])&(df['spot_price']<=df['reserve_ratio']+df['reserve_ratio']*0.05)&(df['reserve_ratio']<=1)
df_filtered = df[reserve_ratio_filter].reset_index()

apy_t_stretches=[]
for APY in np.arange(3, 51, 1):
    min_ts=df_filtered[df_filtered['apy']==APY]['time_stretch'].min()
    max_ts=df_filtered[df_filtered['apy']==APY]['time_stretch'].max()
    apy_t_stretches.append((APY,min_ts,max_ts))

df_filtered

In [ ]:
#df['time_stretch'] = round(df['time_stretch'],1)
#df[(df['apy']==9)&(df['tranche_length']==90)&(df['time_stretch']==9)]
#df_filtered[(df_filtered['apy']==10)]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

plt.subplots(figsize=(12,12))
apys = [apy for apy,min_ts,max_ts in apy_t_stretches]
mean_tss= [(min_ts+max_ts)/2 for apy,min_ts,max_ts in apy_t_stretches]
err_tss= [max_ts-(min_ts+max_ts)/2 for apy,min_ts,max_ts in apy_t_stretches]
plt.yticks(np.arange(0,41, 1))
plt.xticks(np.arange(0,51, 1))
#plt.errorbar(apys, mean_tss, yerr=err_tss, fmt='o', color='black',
#             ecolor='red', elinewidth=3, capsize=0);
plt.scatter(apys,mean_tss,color='black')
plt.grid(True)
plt.title('Suggested Time Stretch vs PT APY', fontsize=14)
plt.xlabel('PT APY', fontsize=14)
plt.ylabel('Time Stretch', fontsize=14)

In [ ]:
apy_t_stretches

In [ ]:
from scipy.optimize import curve_fit

def objective(x,a,b):
    return a/(b*x)

x = apys
y = mean_tss
# curve fit
popt, _ = curve_fit(objective, x, y)
# summarize the parameter values
a, b = popt
print('y = %.5f /( %.5f * x)' % (a, b))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

plt.subplots(figsize=(12,12))
plt.yticks(np.arange(0,41, 1))
plt.xticks(np.arange(0,51, 1))
#plt.errorbar(apys, mean_tss, yerr=err_tss, fmt='o', color='black',
#             ecolor='red', elinewidth=3, capsize=0);
plt.scatter(apys,mean_tss,color='black')
plt.grid(True)
plt.title('Suggested Time Stretch vs PT APY', fontsize=14)
plt.xlabel('PT APY', fontsize=14)
plt.ylabel('Time Stretch', fontsize=14)

x = np.arange(3,51,1)
y = 3.09396 /( 0.02789 * x)
plt.plot(x, y, '--', color="green")

In [ ]:
APY = 1
tranche_length = 30
days_until_maturity = 1
time_stretch = 5
y_reserves = 50
x_reserves = calc_x_reserves(APY,y_reserves,tranche_length,time_stretch)
total_supply=x_reserves
print("required x_reserves: " + str(x_reserves))
g=0.05
t=days_until_maturity/(365*time_stretch)
k=calc_k(y_reserves+total_supply,x_reserves,t)
print("k: " + str(k))
print("k^(1/(1-t)): " + str(k**(1/(1-t))))
max_trade = calc_max_trade(y_reserves+total_supply,x_reserves,t) - .0001
print("max valid trade = {:}".format(max_trade))
(with_fee,without_fee,fee)=calc_out_given_in(max_trade,y_reserves+total_supply,x_reserves,"base",g,t)
print("Total Price after max trade: " + str(without_fee))
unit_price = without_fee/max_trade
print("Unit Price after max trade: " + str(unit_price))
resulting_apy=calc_apy(unit_price,1)
print("resulting apy: " + str(resulting_apy))
(resulting_apy-APY)/APY * 100

In [ ]:
#df = df.drop(columns=['Unnamed: 0']).reset_index()
#df_filtered = df[(df['time_stretch']>=3)]
#df_filtered.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

plt.subplots(figsize=(12,12))

df_filtered = df[(df['time_stretch']==3) & (df['tranche_length']<=180)]

plt.scatter(df_filtered['apy'], df_filtered['max_resulting_apy'], color='orange',label= df_filtered['time_stretch'].iloc[0])

df_filtered = df[(df['time_stretch']==5) & (df['tranche_length']<=180)]

plt.scatter(df_filtered['apy'], df_filtered['max_resulting_apy'], color='green',label= df_filtered['time_stretch'].iloc[0])

df_filtered = df[(df['time_stretch']==10) & (df['tranche_length']<=180)]

plt.scatter(df_filtered['apy'], df_filtered['max_resulting_apy'], color='yellow',label= df_filtered['time_stretch'].iloc[0])

df_filtered = df[(df['time_stretch']==20) & (df['tranche_length']<=180)]

plt.scatter(df_filtered['apy'], df_filtered['max_resulting_apy'], color='blue',label= df_filtered['time_stretch'].iloc[0])

df_filtered = df[(df['time_stretch']==30) & (df['tranche_length']<=180)]
plt.scatter(df_filtered['apy'], df_filtered['max_resulting_apy'], color='red',label= df_filtered['time_stretch'].iloc[0])

df_filtered = df[(df['time_stretch']==40) & (df['tranche_length']<=180)]
plt.scatter(df_filtered['apy'], df_filtered['max_resulting_apy'], color='brown',label= df_filtered['time_stretch'].iloc[0])


plt.title('PT APY Vs Max Resulting APY', fontsize=14)
plt.xlabel('PT APY', fontsize=14)
plt.ylabel('Max Resulting APY', fontsize=14)
plt.grid(True)
plt.legend(title="Time Stretch")
x = np.arange(0,51,1)
y = x
plt.plot(x, y, '--', color="black")
plt.yticks(np.arange(0,101, 2))
plt.xticks(np.arange(0,51, 2))
plt.ylim(0,100)
plt.show()

### This tells me that yield bearing asset APY does have an affect on Max % APY Change

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

plt.subplots(figsize=(12,12))

df_filtered = df[(df['time_stretch']==3) & (df['tranche_length']==30)]
label=(df_filtered['time_stretch'].iloc[0],df_filtered['tranche_length'].iloc[0])
plt.scatter(df_filtered['apy'], df_filtered['max_resulting_apy'], marker='<', color='orange',label=label)
df_filtered = df[(df['time_stretch']==3) & (df['tranche_length']==90)]
label=(df_filtered['time_stretch'].iloc[0],df_filtered['tranche_length'].iloc[0])
plt.scatter(df_filtered['apy'], df_filtered['max_resulting_apy'], marker='o', color='orange',label=label)
df_filtered = df[(df['time_stretch']==3) & (df['tranche_length']==180)]
label=(df_filtered['time_stretch'].iloc[0],df_filtered['tranche_length'].iloc[0])
plt.scatter(df_filtered['apy'], df_filtered['max_resulting_apy'], marker='>', color='orange',label=label)

df_filtered = df[(df['time_stretch']==5) & (df['tranche_length']==30)]
label=(df_filtered['time_stretch'].iloc[0],df_filtered['tranche_length'].iloc[0])
plt.scatter(df_filtered['apy'], df_filtered['max_resulting_apy'], marker='<', color='green',label=label)
df_filtered = df[(df['time_stretch']==5) & (df['tranche_length']==90)]
label=(df_filtered['time_stretch'].iloc[0],df_filtered['tranche_length'].iloc[0])
plt.scatter(df_filtered['apy'], df_filtered['max_resulting_apy'], marker='o', color='green',label=label)
df_filtered = df[(df['time_stretch']==5) & (df['tranche_length']==180)]
label=(df_filtered['time_stretch'].iloc[0],df_filtered['tranche_length'].iloc[0])
plt.scatter(df_filtered['apy'], df_filtered['max_resulting_apy'], marker='>', color='green',label=label)

df_filtered = df[(df['time_stretch']==10) & (df['tranche_length']==30)]
label=(df_filtered['time_stretch'].iloc[0],df_filtered['tranche_length'].iloc[0])
plt.scatter(df_filtered['apy'], df_filtered['max_resulting_apy'], marker='<', color='red',label=label)
df_filtered = df[(df['time_stretch']==10) & (df['tranche_length']==90)]
label=(df_filtered['time_stretch'].iloc[0],df_filtered['tranche_length'].iloc[0])
plt.scatter(df_filtered['apy'], df_filtered['max_resulting_apy'], marker='o', color='red',label=label)
df_filtered = df[(df['time_stretch']==10) & (df['tranche_length']==180)]
label=(df_filtered['time_stretch'].iloc[0],df_filtered['tranche_length'].iloc[0])
plt.scatter(df_filtered['apy'], df_filtered['max_resulting_apy'], marker='>', color='red',label=label)

df_filtered = df[(df['time_stretch']==20) & (df['tranche_length']==30)]
label=(df_filtered['time_stretch'].iloc[0],df_filtered['tranche_length'].iloc[0])
plt.scatter(df_filtered['apy'], df_filtered['max_resulting_apy'], marker='<', color='blue',label=label)
df_filtered = df[(df['time_stretch']==20) & (df['tranche_length']==90)]
label=(df_filtered['time_stretch'].iloc[0],df_filtered['tranche_length'].iloc[0])
plt.scatter(df_filtered['apy'], df_filtered['max_resulting_apy'], marker='o', color='blue',label=label)
df_filtered = df[(df['time_stretch']==20) & (df['tranche_length']==180)]
label=(df_filtered['time_stretch'].iloc[0],df_filtered['tranche_length'].iloc[0])
plt.scatter(df_filtered['apy'], df_filtered['max_resulting_apy'], marker='>', color='blue',label=label)



df_filtered = df[(df['time_stretch']==40) & (df['tranche_length']==30)]
label=(df_filtered['time_stretch'].iloc[0],df_filtered['tranche_length'].iloc[0])
plt.scatter(df_filtered['apy'], df_filtered['max_resulting_apy'], marker='<',color='brown',alpha=.25,label=label)
df_filtered = df[(df['time_stretch']==40) & (df['tranche_length']==90)]
label=(df_filtered['time_stretch'].iloc[0],df_filtered['tranche_length'].iloc[0])
plt.scatter(df_filtered['apy'], df_filtered['max_resulting_apy'], marker='o',color='brown',alpha=.25,label=label)
df_filtered = df[(df['time_stretch']==40) & (df['tranche_length']==180)]
label=(df_filtered['time_stretch'].iloc[0],df_filtered['tranche_length'].iloc[0])
plt.scatter(df_filtered['apy'], df_filtered['max_resulting_apy'], marker='>',color='brown',alpha=.25,label=label)


plt.title('PT APY Vs Max Resulting APY', fontsize=14)
plt.xlabel('PT APY', fontsize=14)
plt.ylabel('Max Resulting APY', fontsize=14)
plt.grid(True)
plt.legend(title="Time Stretch, Tranche Length")
x = np.arange(0,51,1)
y = x
plt.plot(x, y, '--', color="black")
plt.yticks(np.arange(0,101, 2))
plt.ylim(0,100)
plt.xticks(np.arange(0,51, 2))
plt.show()

### This tells me that yield bearing asset APY does have an affect on Max % APY Change


In [ ]:
x = np.arange(0,50,1)
y = x
plt.plot(x, y)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

plt.subplots(figsize=(12,12))

df_filtered = df[df['time_stretch']==5]

plt.scatter(df_filtered['apy'], df_filtered['max_apy_change'], color='green',label= df_filtered['time_stretch'].iloc[0])

df_filtered = df[df['time_stretch']==25]

plt.scatter(df_filtered['apy'], df_filtered['max_apy_change'], color='blue',label= df_filtered['time_stretch'].iloc[0])

df_filtered = df[df['time_stretch']==50]

plt.scatter(df_filtered['apy'], df_filtered['max_apy_change'], color='red',label= df_filtered['time_stretch'].iloc[0])

plt.title('APY Vs Max Percent APY Change', fontsize=14)
plt.xlabel('APY', fontsize=14)
plt.ylabel('Max Percent APY Change', fontsize=14)
plt.grid(True)
plt.legend(title="Time Stretch")
plt.show()

### This tells me that yield bearing asset APY does have an affect on Max % APY Change

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

plt.subplots(figsize=(12,12))

df_filtered = df[df['apy']==1]

plt.scatter(df_filtered['time_stretch'], df_filtered['max_apy_change'], color='green',label= df_filtered['apy'].iloc[0])

df_filtered = df[df['apy']==5]

plt.scatter(df_filtered['time_stretch'], df_filtered['max_apy_change'], color='blue',label= df_filtered['apy'].iloc[0])

df_filtered = df[df['apy']==10]

plt.scatter(df_filtered['time_stretch'], df_filtered['max_apy_change'], color='red',label= df_filtered['apy'].iloc[0])

plt.title('Time Stretch Vs Max Percent APY Change', fontsize=14)
plt.xlabel('Time Stretch', fontsize=14)
plt.ylabel('Max Percent APY Change', fontsize=14)
plt.grid(True)
plt.legend(title="APY")
plt.show()


### This tells me that time stretch does have an affect on Max % APY Change

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

plt.subplots(figsize=(12,12))

df_filtered = df[df['apy']==1]

plt.scatter(df_filtered['tranche_length'], df_filtered['max_apy_change'], color='green',label= df_filtered['apy'].iloc[0])

df_filtered = df[df['apy']==5]

plt.scatter(df_filtered['tranche_length'], df_filtered['max_apy_change'], color='blue', label= df_filtered['apy'].iloc[0])

df_filtered = df[df['apy']==10]

plt.scatter(df_filtered['tranche_length'], df_filtered['max_apy_change'], color='red',label= df_filtered['apy'].iloc[0])

plt.title('Tranche Length Vs Max Percent APY Change', fontsize=14)
plt.xlabel('Tranche Length', fontsize=14)
plt.ylabel('Max Percent APY Change', fontsize=14)
plt.grid(True)
plt.legend(title="APY", loc='upper right')
plt.show()

## This tells me that Tranche Length has virtually no affect on Max % APY Change

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

plt.subplots(figsize=(12,12))

df_filtered = df[df['apy']==1]

plt.scatter(df_filtered['y_reserves'], df_filtered['max_apy_change'], color='green',label= df_filtered['apy'].iloc[0])

df_filtered = df[df['apy']==5]

plt.scatter(df_filtered['y_reserves'], df_filtered['max_apy_change'], color='blue',label= df_filtered['apy'].iloc[0])

df_filtered = df[df['apy']==10]

plt.scatter(df_filtered['y_reserves'], df_filtered['max_apy_change'], color='red',label= df_filtered['apy'].iloc[0])

plt.title('FYT Reserves Vs Max Percent APY Change', fontsize=14)
plt.xlabel('FYT Reserves', fontsize=14)
plt.ylabel('Max Percent APY Change', fontsize=14)
plt.grid(True)
plt.legend(title="APY", loc='upper right')
plt.show()

## This tells me that reserve size has virtually no affect on Max % APY Change

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

plt.subplots(figsize=(12,12))

#df_filtered = df[(df['time_stretch']==5) & (df['y_reserves']==50) & (df['tranche_length']==30)]
df_filtered = df[(df['time_stretch']==5)  & (df['tranche_length']==30)]

plt.scatter(df_filtered['apy'], df_filtered['max_trade_amount'], color='green',label= df_filtered['tranche_length'].iloc[0])

#df_filtered = df[(df['time_stretch']==5) & (df['y_reserves']==500) & (df['tranche_length']==30)]

#plt.scatter(df_filtered['apy'], df_filtered['max_apy_change'], color='blue',label= df_filtered['y_reserves'].iloc[0])

#df_filtered = df[(df['time_stretch']==5) & (df['y_reserves']==1000) & (df['tranche_length']==30)]

#plt.scatter(df_filtered['apy'], df_filtered['max_apy_change'], color='red',label= df_filtered['y_reserves'].iloc[0])

plt.title('APY Vs Max Trade Amount', fontsize=14)
plt.xlabel('APY', fontsize=14)
plt.ylabel('Max Trade Amount', fontsize=14)
plt.grid(True)
plt.legend(title="Tranche Length")
plt.ylim(0,100)
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

plt.subplots(figsize=(12,12))

df_filtered = df[df['time_stretch']==5]

plt.scatter(df_filtered['apy'], df_filtered['min_resulting_fyt_unit_prices'], color='green',label= df_filtered['time_stretch'].iloc[0])

df_filtered = df[df['time_stretch']==25]

plt.scatter(df_filtered['apy'], df_filtered['min_resulting_fyt_unit_prices'], color='blue',label= df_filtered['time_stretch'].iloc[0])

df_filtered = df[df['time_stretch']==50]

plt.scatter(df_filtered['apy'], df_filtered['min_resulting_fyt_unit_prices'], color='red',label= df_filtered['time_stretch'].iloc[0])

plt.title('APY Vs Max Percent APY Change', fontsize=14)
plt.xlabel('APY', fontsize=14)
plt.ylabel('Min Unit FYT Price, fontsize=14)
plt.grid(True)
plt.legend(title="Time Stretch")
plt.show()

### This tells me that yield bearing asset APY does have an affect on Max % APY Change

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

plt.subplots(figsize=(12,12))

df_filtered = df[df['apy']==5]

plt.scatter(df_filtered['time_stretch'], df_filtered['min_resulting_fyt_unit_prices'], color='green',label= df_filtered['apy'].iloc[0])

df_filtered = df[df['apy']==25]

plt.scatter(df_filtered['time_stretch'], df_filtered['min_resulting_fyt_unit_prices'], color='blue',label= df_filtered['apy'].iloc[0])

df_filtered = df[df['apy']==50]

plt.scatter(df_filtered['time_stretch'], df_filtered['min_resulting_fyt_unit_prices'], color='red',label= df_filtered['apy'].iloc[0])

plt.title('APY Vs Max Percent APY Change', fontsize=14)
plt.xlabel('Time Stretch', fontsize=14)
plt.ylabel('Min Unit FYT Price', fontsize=14)
plt.grid(True)
plt.legend(title="APY")
plt.show()



In [ ]:
df.to_csv('apy_change.csv')

In [ ]:
APY = 8.66
term_length = 7
days_until_maturity = 4
time_stretch = 9
y_reserves = 177.52
x_reserves = 2995.64 #calc_x_reserves(APY,y_reserves,term_length-days_until_maturity,time_stretch)
print("(APY,y_reserves,days_until_maturity,time_stretch): "+str((APY,y_reserves,days_until_maturity,time_stretch)))
print("required x_reserves: " + str(x_reserves))
total_supply=x_reserves
price=calc_price(x_reserves,y_reserves,total_supply,days_until_maturity,time_stretch)
print("resulting price: " + str(price))
print("check apy: " + str(calc_apy(price,days_until_maturity)))

In [ ]:
x_reserves = 1856051.71
y_reserves = 1425357.41
total_supply = x_reserves+1425357.41
APY = 11.4
unit_price = 0.9728
t = 85/(365*9)

(x_reserves/(y_reserves+total_supply))**t

In [ ]:
3284587796090403818248754/10**18


In [ ]:
1856051.71 + 1425357.41

In [ ]:
3.09396 /( 0.02789 * 20)

In [ ]:
5.11436 /( 0.04610 * 10)